# 보충 자료

## 01_linear_regression.ipynb의 주요 내용

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

%matplotlib inline

In [2]:
# Read the data
data = pd.read_csv('data/bikeshare.csv')

# Year와 Month를 추출
datetime = pd.DatetimeIndex(data['datetime'])
data['year'] = datetime.year
data['month'] = datetime.month
data['hour'] = datetime.hour

# "count" is a method, so it's best to name that column something else
data.rename(columns={'count':'total'}, inplace=True)

# Handling 'season' variable
season_dummies = pd.get_dummies(data.season, prefix='season')
season_dummies.drop(season_dummies.columns[0], axis=1, inplace=True)
data = pd.concat([data, season_dummies], axis=1)

# Add derivative variable "daytime"
data['daytime'] = ((data.hour > 6) & (data.hour < 21)).astype(int)

# Handling 'hour' variable
hour_dummies = pd.get_dummies(data.hour, prefix='hour')
hour_dummies.drop(hour_dummies.columns[0], axis=1, inplace=True)
data = pd.concat([data, hour_dummies], axis=1)

## 선형회귀모델을 학습하는 함수를 조금 수정하였습니다.
다음을 포함하는 dictionary를 출력하는 함수로 변경하였습니다.
- 각 변수에 대응하는 계수들(coefficients)과 intercept
- Train set에서의 RMSE, R^2
- Test set에서의 RMSE, R^2

In [4]:
# Define a function that accepts a list of features and
# returns coefficients, intercept, training RMSE/R^2 and testing RMSE/R^2
def train_test_linreg(d, feature_cols):
    X = d[feature_cols]
    Y = d.total
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=123)
    model = LinearRegression()
    model.fit(X_train, Y_train)
    
    # Make series using selected features and corresponding coefficients
    formula = pd.Series(model.coef_, index = feature_cols)
    
    # Save intercept
    intercept = model.intercept_
    
    # Calculate training RMSE and testing RMSE
    Y_pred_train = model.predict(X_train)
    Y_pred_test = model.predict(X_test)
    rmse_train = np.sqrt(metrics.mean_squared_error(Y_train, Y_pred_train))
    rmse_test = np.sqrt(metrics.mean_squared_error(Y_test, Y_pred_test))
    
    # Calculate training R-square and testing R-square
    rsquared_train = model.score(X_train, Y_train)
    rsquared_test = model.score(X_test, Y_test)
    
    # Make result dictionary
    result={'formula':formula, 'intercept':intercept, 'rmse_train':rmse_train, 'rmse_test':rmse_test,
           'rsquared_train':rsquared_train, 'rsquared_test':rsquared_test}
    
    return result

In [5]:
# Hour에 대한 binary dummy variable만 이용하여 선형회귀모델을 학습
hour_cols = list(data.columns[data.columns.str.startswith('hour_')])
result = train_test_linreg(data, hour_cols)

In [6]:
# Check coefficients, intercept, training RMSE/R^2 and testing RMSE/R^2
result

{'formula': hour_1     -22.580917
 hour_2     -32.757474
 hour_3     -44.209704
 hour_4     -49.961957
 hour_5     -36.711049
 hour_6      16.801172
 hour_7     159.656510
 hour_8     309.943473
 hour_9     160.149618
 hour_10    121.193570
 hour_11    147.090587
 hour_12    202.425259
 hour_13    200.110370
 hour_14    195.261156
 hour_15    195.486156
 hour_16    266.890255
 hour_17    419.952457
 hour_18    379.068371
 hour_19    264.786324
 hour_20    173.622659
 hour_21    125.548656
 hour_22     77.631922
 hour_23     34.730308
 dtype: float64,
 'intercept': 56.263843648208336,
 'rmse_train': 124.92842235488433,
 'rmse_test': 128.47511657303033,
 'rsquared_train': 0.5263145338683541,
 'rsquared_test': 0.4919246495057706}

## Ridge regression & Lasso regression
### 두 모델의 공통점
- **Regularization**: 모델 계수가 커지는 것에 대한 penalty를 부여함으로써 모델의 overfitting(과적합)을 방지
- 기본적인 multiple linear regression (다중선형회귀분석) 은 변수 간의 [다중공산성(multicollinearity)](https://ko.wikipedia.org/wiki/%EB%8B%A4%EC%A4%91%EA%B3%B5%EC%84%A0%EC%84%B1)에 의해 성능이 하락하는데, 이 두 회귀모델은 이에 대해 대처할 수 있는 모델
- 모델의 parameter(모수)가 존재: 계수 크기에 대한 penalty를 얼마나 줄 것인가 (**alpha**)
- alpha가 0이면 단순 다중선형회귀모델과 일치


### Lasso regression의 강점
- Lasso regression은 ridge regression과는 달리 특정 변수의 계수를 0으로 만들어줍니다. 특정 변수의 계수가 0이 아니라는 것은 **lasso regression 모델이 그 변수를 선택**했다고 볼 수 있습니다.
- Lasso regression은 모든 변수가 선택되는 것이 아니라는 점에서 **sparse model** (희소모델)


두 모델을 적용하기에 앞서 다음과 같은 데이터 전처리를 다시 실시하였습니다.
- X에서 가능한 모든 변수를 사용하여, 모델의 성능이 어떻게 나오는지 파악
- 제거한 변수: datetime (수치형 변수가 아니며, year/month/hour로 이미 분리됨),casual & registered (타겟변수인 'total'과 함께 움직이는 변수), total (타겟 변수)

In [7]:
from sklearn.linear_model import Ridge, Lasso

In [8]:
def train_test_ridge(data, alpha_value):
    X = data.drop(['datetime','casual','registered','total', 'season', 'hour'], axis = 1)
    Y = data.total
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=123)
    model = Ridge(alpha = alpha_value)
    model.fit(X_train, Y_train)
    
    # Make series using selected features and corresponding coefficients
    formula = pd.Series(model.coef_, index = list(X.columns.values))
    
    # Save intercept
    intercept = model.intercept_
    
    # Calculate training RMSE and testing RMSE
    Y_pred_train = model.predict(X_train)
    Y_pred_test = model.predict(X_test)
    rmse_train = np.sqrt(metrics.mean_squared_error(Y_train, Y_pred_train))
    rmse_test = np.sqrt(metrics.mean_squared_error(Y_test, Y_pred_test))
    
    # Calculate training R-square and testing R-square
    rsquared_train = model.score(X_train, Y_train)
    rsquared_test = model.score(X_test, Y_test)
    
    # Make result dictionary
    result={'formula':formula, 'intercept':intercept, 'rmse_train':rmse_train, 'rmse_test':rmse_test,
           'rsquared_train':rsquared_train, 'rsquared_test':rsquared_test}
    
    return result

In [11]:
# Ridge regression 학습 및 테스트 (alpha = 0.1)
result = train_test_ridge(data, 5)

In [12]:
result

{'formula': holiday        -0.241201
 workingday      3.557334
 weather       -24.250728
 temp            2.657642
 atemp           2.693038
 humidity       -0.731705
 windspeed      -0.539184
 year           86.444968
 month           8.290617
 season_2       20.127684
 season_3      -16.690652
 season_4       -7.824809
 daytime       191.240046
 hour_1        -23.346823
 hour_2        -30.521447
 hour_3        -43.572011
 hour_4        -46.730807
 hour_5        -26.958390
 hour_6         27.121898
 hour_7        -26.382880
 hour_8        123.076170
 hour_9        -34.180309
 hour_10       -83.987526
 hour_11       -62.543739
 hour_12       -16.971829
 hour_13       -23.317104
 hour_14       -36.655135
 hour_15       -33.326099
 hour_16        33.768118
 hour_17       190.191174
 hour_18       151.853058
 hour_19        47.584876
 hour_20       -37.868730
 hour_21       102.251093
 hour_22        64.681551
 hour_23        27.221663
 dtype: float64,
 'intercept': -173899.69805504192,
 

In [13]:
def train_test_lasso(data, alpha_value):
    X = data.drop(['datetime','casual','registered','total', 'season', 'hour'], axis = 1)
    Y = data.total
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=123)
    model = Lasso(alpha = alpha_value)
    model.fit(X_train, Y_train)
    
    # Make series using selected features and corresponding coefficients
    formula = pd.Series(model.coef_, index = list(X.columns.values))
    
    # Save intercept
    intercept = model.intercept_
    
    # Calculate training RMSE and testing RMSE
    Y_pred_train = model.predict(X_train)
    Y_pred_test = model.predict(X_test)
    rmse_train = np.sqrt(metrics.mean_squared_error(Y_train, Y_pred_train))
    rmse_test = np.sqrt(metrics.mean_squared_error(Y_test, Y_pred_test))
    
    # Calculate training R-square and testing R-square
    rsquared_train = model.score(X_train, Y_train)
    rsquared_test = model.score(X_test, Y_test)
    
    # Make result dictionary
    result={'formula':formula, 'intercept':intercept, 'rmse_train':rmse_train, 'rmse_test':rmse_test,
           'rsquared_train':rsquared_train, 'rsquared_test':rsquared_test}
    
    return result

In [18]:
# Lasso regression 학습 및 테스트 (alpha = 0.1)
result = train_test_lasso(data, 3)

In [19]:
result

{'formula': holiday        -0.000000
 workingday      0.000000
 weather        -9.913521
 temp            1.266311
 atemp           4.476017
 humidity       -1.327402
 windspeed      -0.224684
 year           71.943534
 month           6.761166
 season_2        5.685929
 season_3       -1.415003
 season_4        0.000000
 daytime       157.830886
 hour_1         -0.000000
 hour_2         -0.000000
 hour_3         -0.000000
 hour_4         -0.000000
 hour_5         -0.000000
 hour_6          0.000000
 hour_7         -0.000000
 hour_8         76.457096
 hour_9         -0.000000
 hour_10        -0.000000
 hour_11        -0.000000
 hour_12        -0.000000
 hour_13        -0.000000
 hour_14        -0.000000
 hour_15        -0.000000
 hour_16         0.000000
 hour_17       130.447777
 hour_18        87.489301
 hour_19         0.000000
 hour_20        -0.000000
 hour_21        16.864350
 hour_22         0.000000
 hour_23         0.000000
 dtype: float64,
 'intercept': -144704.9729606603,
 '

### 파악할 부분
- Ridge regression과 Lasso regression의 결과와 단순선형회귀모델의 결과를 비교해보세요.
- 위의 Ridge regression과 Lasso regression에서 alpha값을 변형해가면서 결과가 달라지는지 살펴보세요.